# Z2Pack

## Setup

In [2]:
%%capture
%%bash
pip install z2pack

In [4]:
import tbmodels
import z2pack
import lzma

Uncompress HDF5 file

In [5]:
with lzma.open('wte2_soc.tar.xz') as fin, open('wte2_soc.hdf5', 'wb') as fout:
    fout.write(fin.read())

## Calculate Chern number for WTe<sub>2</sub> Weyl node 

In [6]:
model = tbmodels.io.load('wte2_soc.hdf5')

/home/greschd/.virtualenvs/tools_intro/lib/python3.6/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


Calculate the Wannier charge centers on a sphere of radius $r=0.005$ around $\mathbf{k}=(0.1203, 0.05232, 0)$:

## $\mathbb{Z}_2$ Invariant

The procedure for calculating the $\mathbb{Z}_2$ invariant is the same, except due to the surface should go only across half the Brillouin zone (between two Kramers-degenerate lines).

In this example, we calculate the $\mathbb{Z}_2$ invariant for a specific set of constants in the Kane-Mele Hamiltonian:

In [29]:
import numpy as np
from numpy import sin, cos, kron, sqrt

In [30]:
identity = np.identity(2, dtype=complex)
pauli_x = np.array([[0, 1], [1, 0]], dtype=complex)
pauli_y = np.array([[0, -1j], [1j, 0]], dtype=complex)
pauli_z = np.array([[1, 0], [0, -1]], dtype=complex)

In [31]:
def kane_mele_hamiltonian(k):
    # Fixed constants:
    t = 1
    lambda_v = 0.1
    lambda_R = 0.05
    lambda_SO = 0.06

    k = np.array(k) * 2 * np.pi
    kx, ky = k
    # change to reduced coordinates
    x = (kx - ky) / 2
    y = (kx + ky) / 2
    return (t * (1 + 2 * cos(x) * cos(y)) * kron(pauli_x, identity) +
            lambda_v * kron(pauli_z, identity) +
            lambda_R * (1 - cos(x) * cos(y)) * kron(pauli_y, pauli_x) +
            -sqrt(3) * lambda_R * sin(x) * sin(y) * kron(pauli_y, pauli_y) +
            2 * t * cos(x) * sin(y) * kron(pauli_y, identity) + lambda_SO *
            (2 * sin(2 * x) - 4 * sin(x) * cos(y)) * kron(pauli_z, pauli_z) +
            lambda_R * cos(x) * sin(y) * kron(pauli_x, pauli_x) +
            -sqrt(3) * lambda_R * sin(x) * cos(y) * kron(pauli_x, pauli_y))